In [5]:
#Import Libraries and print their versions
%run /content/drive/MyDrive/Bayesian\ path\ analysis/imports.py
#%run imports.py

Python version:     3.11.12
pymc version:       5.21.2
numpy version:      2.0.2
pandas version:     2.2.2
arviz version:      0.21.0
matplotlib version: 3.10.0


In [6]:
#%run /content/drive/MyDrive/Bayesian\ path\ analysis/model_B02.py
%run model_B02.py

In [25]:
def calculate_odds_ratios(idata, var_names=("intercept", "effect", "level_effects")):
    """
    Calculates the odds ratios and their 95% HDIs for specified variables.

    Args:
        idata (az.InferenceData): InferenceData object.
        var_names (tuple, optional): Names of the parameters for which to
                                    calculate odds ratios.
                                    Defaults to ("intercept", "effect", "level_effects").

    Returns:
        dict: A dictionary where keys are variable names and values are Pandas
              DataFrames with mean, lower HDI, and upper HDI for the odds ratio.
              Returns an empty dictionary if there's an error.
    """

    odds_ratios_results = {}
    for var_name in var_names:
        try:
            posterior_samples = idata.posterior[var_name].values.flatten()
            # For 'level_effects', calculate odds ratios for differences
            if var_name == "level_effects":
                or_values = np.exp(np.diff(posterior_samples, axis=0))
            else:
                or_values = np.exp(posterior_samples)
            hdi = az.hdi(or_values, hdi_prob=0.95)
            mean_or = np.mean(or_values)

            or_summary = pd.DataFrame({
                "mean": [mean_or],
                "hdi_lower": [hdi[0]],
                "hdi_upper": [hdi[1]]
            })
            odds_ratios_results[var_name] = or_summary

        except KeyError:
            print(f"Variable '{var_name}' not found in posterior.")
        except Exception as e:
            print(f"Error calculating odds ratio for '{var_name}': {e}")

    return odds_ratios_results


In [33]:
def run_and_summarize(data, model_func, label="b02", output_dir="../results"):
    """
    Runs the model, summarizes results, saves outputs: idata, summary, metrics, plots.
    Works for B01 or B02 depending on the model_func provided.
    """
    os.makedirs(output_dir, exist_ok=True)

    model, idata, waic, loo, bic = model_func(data["predictor"], data["outcome"])

    # Save InferenceData
    idata_path = os.path.join(output_dir, f"idata_{label}.nc")
    idata.to_netcdf(idata_path)

    # Determine var_names based on model
    try:
        # Check if variables like "effect" and "sd_fluctuation" exist (i.e. B02)
        var_names = ["intercept", "effect", "sd_fluctuation", "level_effects", "p"]
        _ = idata.posterior["effect"]
    except KeyError:
        # B01 fallback
        var_names = ["intercept", "level_effects", "p"]

    # Save Summary
    summary = az.summary(idata, var_names=var_names)
    summary_path = os.path.join(output_dir, f"survey_summary_{label}.csv")
    summary.to_csv(summary_path)

    # Calculate and save Odds Ratios summaries
    or_summaries = calculate_odds_ratios(idata, var_names=var_names)
    for var_name, or_summary in or_summaries.items():
        if not or_summary.empty:
            or_summary_path = os.path.join(output_dir,
                                           f"survey_or_summary_{label}_{var_name}.csv")
            or_summary.to_csv(or_summary_path)
        else:
            print(f"Odds ratio calculation failed for {var_name}. Not saving.")


    # Save WAIC, LOO, BIC as a markdown file
    metrics_path = os.path.join(output_dir, f"survey_model_metrics_{label}.md")
    with open(metrics_path, "w") as f:
        f.write(f"## Model {label.upper()} Metrics\n\n")
        f.write("### WAIC\n```\n")
        f.write(waic.to_string() + "\n```\n")
        f.write("### LOO\n```\n")
        f.write(loo.to_string() + "\n```\n")
        f.write(f"### BIC\n```\n\nBIC = {bic:.2f}\n```")

    # Save Plots
    trace_plot_path = os.path.join(output_dir, f"survey_trace_{label}.png")
    az.plot_trace(idata, var_names=[vn for vn in var_names if vn != "p"])
    plt.tight_layout()
    plt.savefig(trace_plot_path, dpi=300)
    plt.close()

    posterior_plot_path = os.path.join(output_dir, f"survey_posterior_p_{label}.png")
    az.plot_posterior(idata, var_names=["p"])
    plt.tight_layout()
    plt.savefig(posterior_plot_path, dpi=300)
    plt.close()

    ppc_plot_path = os.path.join(output_dir, f"survey_ppc_{label}.png")
    az.plot_ppc(idata)
    plt.tight_layout()
    plt.savefig(ppc_plot_path, dpi=300)
    plt.close()

    print(f"✅ Saved all outputs for model {label.upper()} in '{output_dir}' folder.")

    return idata, waic, loo, bic


### Tesing the model using the survey dataset

In [9]:
# Survey Data Generation
#%run /content/drive/MyDrive/Bayesian\ path\ analysis/code_snippets.py
%run code_snippets.py

25-26/77-78 27-28/79-80 29-30/81-82 31-32/83-84 33-34/85-86 35-36/87-88 37-38/89-90 39-40/91-92 41-42/93-94 43-44/95-96


In [6]:
d.columns

Index(['Unnamed: 0', 'Idő', 'Befejezés', 'Forrás',
       'Változtatott-e önszántából munkahelyet 2022. január 1. óta?',
       'Rendezze sorrendbe fontosság szerint az alábbi, a munkához kapcsolódó értékeket. A sorba rendezéshez használjon csúszkát, ne a nyilakat!\nMit tartott elsősorban, másodsorban stb. fontosnak, amikor munkahelyet változtatott? x Kevés stresszel járó munka',
       'Rendezze sorrendbe fontosság szerint az alábbi, a munkához kapcsolódó értékeket. A sorba rendezéshez használjon csúszkát, ne a nyilakat!\nMit tartott elsősorban, másodsorban stb. fontosnak, amikor munkahelyet változtatott? x Hosszú szabadság',
       'Rendezze sorrendbe fontosság szerint az alábbi, a munkához kapcsolódó értékeket. A sorba rendezéshez használjon csúszkát, ne a nyilakat!\nMit tartott elsősorban, másodsorban stb. fontosnak, amikor munkahelyet változtatott? x Szakmai fejlődési lehetőség',
       'Rendezze sorrendbe fontosság szerint az alábbi, a munkához kapcsolódó értékeket. A sorba rende

In [10]:
# Rename columns to C999 format
new_columns = {old_col: f'C{i:03d}' for i, old_col in enumerate(d.columns)}
d = d.rename(columns=new_columns)

In [11]:
d.columns

Index(['C000', 'C001', 'C002', 'C003', 'C004', 'C005', 'C006', 'C007', 'C008',
       'C009',
       ...
       'C102', 'C103', 'C104', 'C105', 'C106', 'C107', 'C108', 'C109', 'C110',
       'C111'],
      dtype='object', length=112)

In [12]:
#Merge the predictor columns
d["I_I"] = merge_columns(d["C027"], d["C079"])
print("\nDataframe with merged \"I_I\" columns (first 5 rows of relevant columns):")
print(d[["C027", "C079", "I_I"]].head())


Dataframe with merged "I_I" columns (first 5 rows of relevant columns):
              C027             C079              I_I
0              NaN       Egyetértek       Egyetértek
1              NaN       Egyetértek       Egyetértek
2  Nem értek egyet              NaN  Nem értek egyet
3              NaN  Nem értek egyet  Nem értek egyet
4              NaN  Nem értek egyet  Nem értek egyet


In [13]:
#Prepare the outcome variable
print("\nUnique values in the outcome columns (C004):")
print(d["C004"].unique())


Unique values in the outcome columns (C004):
['Nem' 'Igen']


In [14]:
d["outcome_binary"] = d["C004"].apply(lambda x: 1 if x == "Igen" else 0)
print("\nFirst few rows with binary outcome:")
print(d[["C004", "outcome_binary"]].head())


First few rows with binary outcome:
   C004  outcome_binary
0   Nem               0
1   Nem               0
2  Igen               1
3   Nem               0
4   Nem               0


In [15]:
#Prepare the predictor variable (I_I) - Dichotomize into agree/disagree
print("\nUnique values in the merged 'I_I' column:")
print(d["I_I"].unique())


Unique values in the merged 'I_I' column:
['Egyetértek' 'Nem értek egyet' 'Teljesen egyetértek'
 'Egyáltalán nem értek egyet']


In [17]:
#Change values of the likert scale to english
mapping = {'Egyetértek': 'I Agree', 'Egyáltalán nem értek egyet': 'I Completely Disagree', 'Nem értek egyet': 'I Do Not Agree', 'Teljesen egyetértek': 'I totally Agree'}
d["I_I"] = d["I_I"].map(mapping)
print("\nUnique values in the 'I_I' column after mapping:")
print(d["I_I"].unique())


Unique values in the 'I_I' column after mapping:
['I Agree' 'I Do Not Agree' 'I totally Agree' 'I Completely Disagree']


In [18]:
d.describe()

,C005,C006,C007,C008,C009,C010,C011,C012,C013,C014,...,C068,C069,C070,C071,C072,C073,C074,C075,C076,outcome_binary
count,125.000000,125.000000,125.000000,125.00000,125.000000,125.000000,125.000000,125.000000,125.000000,125.000000,...,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,333.000000
mean,14.648000,10.200000,16.688000,12.88800,13.352000,14.024000,16.904000,10.472000,12.976000,11.760000,...,7.817308,7.456731,9.658654,8.533654,5.875000,4.687500,8.879808,8.721154,6.408654,0.375375
std,4.882913,5.840625,3.798166,4.94847,4.924029,3.766285,2.613316,4.363662,3.670862,4.135605,...,4.417279,4.014232,4.655662,4.684110,4.111225,3.955481,5.625166,5.349535,6.019399,0.484948
min,2.000000,1.000000,4.000000,1.00000,1.000000,3.000000,9.000000,1.000000,4.000000,1.000000,...,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,11.000000,5.000000,15.000000,10.00000,11.000000,12.000000,15.000000,7.000000,10.000000,9.000000,...,4.000000,5.000000,6.000000,5.000000,3.000000,2.000000,3.000000,4.000000,1.000000,0.000000
50%,16.000000,10.000000,18.000000,14.00000,14.000000,15.000000,17.000000,11.000000,13.000000,11.000000,...,8.000000,7.000000,9.000000,7.000000,5.000000,4.000000,9.000000,9.000000,4.000000,0.000000
75%,19.000000,15.000000,20.000000,17.00000,17.000000,17.000000,19.000000,13.000000,16.000000,14.000000,...,11.000000,10.000000,13.250000,12.000000,8.000000,7.000000,14.000000,13.000000,11.000000,1.000000
max,20.000000,20.000000,20.000000,20.00000,20.000000,20.000000,20.000000,19.000000,20.000000,20.000000,...,19.000000,19.000000,20.000000,19.000000,17.000000,20.000000,20.000000,20.000000,20.000000,1.000000


In [19]:
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333 entries, 0 to 332
Columns: 114 entries, C000 to outcome_binary
dtypes: float64(40), int64(1), object(73)
memory usage: 296.7+ KB


In [20]:
d.isna().sum()

,0
C000,0
C001,0
C002,0
C003,0
C004,0
...,...
C109,24
C110,25
C111,0
I_I,0


In [21]:
likert_mapping = {
    "I Completely Disagree": 1,
    "I Do Not Agree": 2,
    "I Agree": 3,
    "I totally Agree": 4
}
d["I_I_numeric"] = d["I_I"].map(likert_mapping)

In [27]:
d.describe()

,C005,C006,C007,C008,C009,C010,C011,C012,C013,C014,...,C069,C070,C071,C072,C073,C074,C075,C076,outcome_binary,I_I_numeric
count,125.000000,125.000000,125.000000,125.00000,125.000000,125.000000,125.000000,125.000000,125.000000,125.000000,...,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,333.000000,333.000000
mean,14.648000,10.200000,16.688000,12.88800,13.352000,14.024000,16.904000,10.472000,12.976000,11.760000,...,7.456731,9.658654,8.533654,5.875000,4.687500,8.879808,8.721154,6.408654,0.375375,2.729730
std,4.882913,5.840625,3.798166,4.94847,4.924029,3.766285,2.613316,4.363662,3.670862,4.135605,...,4.014232,4.655662,4.684110,4.111225,3.955481,5.625166,5.349535,6.019399,0.484948,0.842536
min,2.000000,1.000000,4.000000,1.00000,1.000000,3.000000,9.000000,1.000000,4.000000,1.000000,...,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000
25%,11.000000,5.000000,15.000000,10.00000,11.000000,12.000000,15.000000,7.000000,10.000000,9.000000,...,5.000000,6.000000,5.000000,3.000000,2.000000,3.000000,4.000000,1.000000,0.000000,2.000000
50%,16.000000,10.000000,18.000000,14.00000,14.000000,15.000000,17.000000,11.000000,13.000000,11.000000,...,7.000000,9.000000,7.000000,5.000000,4.000000,9.000000,9.000000,4.000000,0.000000,3.000000
75%,19.000000,15.000000,20.000000,17.00000,17.000000,17.000000,19.000000,13.000000,16.000000,14.000000,...,10.000000,13.250000,12.000000,8.000000,7.000000,14.000000,13.000000,11.000000,1.000000,3.000000
max,20.000000,20.000000,20.000000,20.00000,20.000000,20.000000,20.000000,19.000000,20.000000,20.000000,...,19.000000,20.000000,19.000000,17.000000,20.000000,20.000000,20.000000,20.000000,1.000000,4.000000


In [28]:
#Prepare data for the Bayesian model
y = d["outcome_binary"].values
X = d["I_I_numeric"].values

In [29]:
d_model = pd.DataFrame({
        "predictor": X,
        "outcome": y
    })
d_model.head()

,predictor,outcome
0,3,0
1,3,0
2,2,1
3,2,0
4,2,0


In [34]:
# Run and save results for B02 for survey data
print("\nResults for B02 Model on survey data:")
idata_b02_s, waic_b02_s, loo_b02_s, bic_b02_s = run_and_summarize(d_model, ordinal_predictor_binary_outcome_model, label="b02")


Results for B02 Model on survey data:


Output()

ERROR:pymc.stats.convergence:There were 58 divergences after tuning. Increase `target_accept` or reparameterize.


Output()

/usr/local/lib/python3.11/dist-packages/arviz/stats/diagnostics.py:596: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
/usr/local/lib/python3.11/dist-packages/arviz/stats/diagnostics.py:991: RuntimeWarning: invalid value encountered in scalar divide
  varsd = varvar / evar / 4
/usr/local/lib/python3.11/dist-packages/arviz/stats/density_utils.py:488: UserWarning: Your data appears to have a single value or no finite values
  warnings.warn("Your data appears to have a single value or no finite values")
/usr/local/lib/python3.11/dist-packages/arviz/plots/plot_utils.py:270: UserWarning: rcParams['plot.max_subplots'] (40) is smaller than the number of variables to plot (333) in plot_posterior, generating only 40 plots
  warnings.warn(


✅ Saved all outputs for model B02 in 'results' folder.


In [35]:
from google.colab import files
!zip -r survey_results_b02.zip results/
files.download("survey_results_b02.zip")

  adding: results/ (stored 0%)
  adding: results/survey_trace_b02.png (deflated 4%)
  adding: results/survey_or_summary_b02_intercept.csv (deflated 11%)
  adding: results/survey_summary_b02.csv (deflated 93%)
  adding: results/survey_ppc_b02.png (deflated 23%)
  adding: results/survey_or_summary_b02_p.csv (deflated 9%)
  adding: results/survey_posterior_p_b02.png (deflated 19%)
  adding: results/survey_or_summary_b02_effect.csv (deflated 13%)
  adding: results/idata_b02.nc (deflated 23%)
  adding: results/survey_or_summary_b02_sd_fluctuation.csv (deflated 11%)
  adding: results/survey_model_metrics_b02.md (deflated 77%)
  adding: results/survey_or_summary_b02_level_effects.csv (deflated 9%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [36]:
def calculate_bayesian_odds_ratio(idata, effect_var_name="effect"):
    """
    Calculates the mean and 95% Highest Density Interval (HDI)
    for the odds ratio from a Bayesian model.

    Args:
        idata: ArviZ InferenceData object containing the posterior samples.
        effect_var_name: The name of the variable representing the effect
                         on the log-odds scale (default: "effect").

    Returns:
        A tuple containing:
        - mean_odds_ratio: The mean odds ratio.
        - hdi_odds_ratio:  A dictionary containing the 95% HDI for the odds ratio.
                          Returns (None, None) if there's an error.
    """
    try:
        # Extract posterior samples for the effect
        posterior_effect_samples = idata.posterior[effect_var_name].values.flatten()
    except KeyError:
        print(f"Error: Variable '{effect_var_name}' not found in idata.posterior.")
        return None, None

    # Calculate odds ratios
    posterior_odds_ratios = np.exp(posterior_effect_samples)

    # Calculate mean odds ratio
    mean_odds_ratio = np.mean(posterior_odds_ratios)

    # Calculate 95% HDI for odds ratio
    hdi_odds_ratio = az.hdi(posterior_odds_ratios, hdi_prob=0.95)

    return mean_odds_ratio, hdi_odds_ratio

# Load the InferenceData object
# Assuming idata_b02 is already loaded or you have the path to it
idata_b02 = az.from_netcdf("/content/results/idata_b02.nc")# Replace with your actual path

# Calculate and print the odds ratio and its HDI
mean_or, hdi_or = calculate_bayesian_odds_ratio(idata_b02)

if mean_or is not None:
    print(f"Bayesian Model Odds Ratio: Mean = {mean_or:.2f}, 95% HDI = {hdi_or}")

Bayesian Model Odds Ratio: Mean = 0.79, 95% HDI = [0.32666381 1.40324828]
